In [1]:
import pandas as pd

In [4]:
d = pd.read_csv("Tables/Postdoc_data_2016.csv")
list(d.columns.values)

['Institution',
 'Title',
 'Annual Salary',
 'FTE',
 'APPT FTR BASIS',
 'APPT FRACTION',
 'AMOUNT OF SALARY PAID FROM GENERAL FUND',
 '%',
 'FTE.1',
 'Time',
 'PAY_FRCTN',
 'Last Name',
 'First Names',
 'Department',
 'FT/PT',
 'Contract Length',
 'Degree',
 'Highest Education Level',
 'Type',
 'Effective Date',
 'Hire Date']

## 1. Ethnea

In [7]:
#Create the two new columns (sex and ethnicity) and fill them with True/False depending on the presence of last/first name
d["Ethnea"] = d["Last Name"].notnull()
d["Genni"] = d["First Names"].notnull()

In [8]:
from urllib.request import urlopen
import ast

In [9]:
#Set up querry
for i in range(len(d)):
    url = "http://abel.lis.illinois.edu/cgi-bin/ethnea/search.py?Fname=FIRSTNAME&Lname=LASTNAME&format=json"
    if d["Genni"][i] == True or d["Ethnea"][i] == True:
        if d["Genni"][i] == True:
            fname = d["First Names"][i]
            if (' ' in fname) == True:
                fname = fname.split()
                fname = "%20".join(fname)
        else:
            fname = ""
        if d["Ethnea"][i] == True:
            lname = d["Last Name"][i]
            if (' ' in lname) == True:
                lname = lname.split()
                lname = "%20".join(lname)
        else:
            lname = ""
        url = url.replace('FIRSTNAME',fname).replace('LASTNAME',lname)
#Perform querry
        data = ast.literal_eval(urlopen(url).read().rstrip().decode('UTF-8'))
        d.loc[i, "Ethnea"] = data['Ethnea']
        d.loc[i, "Genni"] = data['Genni']

In [26]:
 d.to_csv("Tables/Gendered.csv")

In [37]:
d = pd.read_csv("Tables/Gendered.csv")

## 2. Salary annualization

In [38]:
import math

In [39]:
d["AdjSalary"] = 1
for i in range(len(d)):
    d.loc[i,"AdjSalary"] = d["Annual Salary"][i].replace('$',"").replace(',',"").replace('*',"")
    frq = 1
    if math.isnan(d["%"][i]) == False:
        frq = float(d["%"][i])
    if math.isnan(d["FTE"][i]) == False:
        frq = float(d["FTE"][i])
    if math.isnan(d["FTE.1"][i]) == False:
        frq = float(d["FTE.1"][i])
    if math.isnan(d["Time"][i]) == False:
        frq = float(d["Time"][i])
    if math.isnan(d["PAY_FRCTN"][i]) == False:
        frq = float(d["PAY_FRCTN"][i])
    if frq > 1:
        d.loc[i,"AdjSalary"] = float(d.loc[i,"AdjSalary"]) / (frq/100)
    if frq <= 1:
        d.loc[i,"AdjSalary"] = float(d.loc[i,"AdjSalary"]) / frq

## 3. Total number of postdocs

In [42]:
d["PostdocNum"] = 0.1
univ = list(set(d["Institution"]))
for i in range(len(univ)):
    num = sum([1 for j in list(d["Institution"]) if j==univ[i]])
    ind = d.loc[d["Institution"] == univ[i]].index
    d.loc[ind, "PostdocNum"] = num

## 4. Postdoc titles

In [44]:
#Turn 91 different titles into a handfull of defining words
words = {'Fellow':["fellow"],
        'Associate': ["asso", "aso"],
        'Researcher' : ["res","not_resident"],
         'Scholar' : ["scholar"],
         'Intern' : ["intern"],
         'Trainee' : ["trainee"],
         'Teaching' : ["teach"],
         'Senior' : ["senior", "sr"],
         'Assistant' : ["asst"],
         'Clinical' : ["clinical"], 
        'Assistant' : ["asst", "assistant"],
        'Faculty' : ["fac"]}

for word in words.keys():
    d[word] = 0

In [46]:
import re

In [47]:
for i in range(len(d)):
    target = d.loc[i,"Title"].lower()
    for word in words.keys():
        search_result = 0
        if len(words[word]) == 1:
            search_result = len(re.findall(words[word][0], target))
        if len(words[word]) > 1:
            if "not_" in words[word][1]:
                word1 = re.sub("not_","",words[word][1])
                search_result = len(re.findall(words[word][0], target)) - len(re.findall(word1, target))
            else:
                search_result = len(re.findall(words[word][0], target)) + len(re.findall(words[word][1], target))
        if search_result>0:
            d.loc[i, word] = 1

## 5. NIH grants

In [52]:
univ_info = pd.read_csv("Univ_info.csv")

In [53]:
d["NIH_grants"] = ""
d["NIH_order"] = 0
for i in range(len(d)):
    for j in range(len(univ_info)):
        if univ_info.loc[j, "University_FoiA"].strip() == d.loc[i, "Institution"].strip():
            d.loc[i, "NIH_grants"] = univ_info.loc[j, "2017_NIH"]
            d.loc[i, "NIH_order"] = univ_info.loc[j,"NIH_top"]

In [55]:
#Final mega-table
list(d.columns.values)

['Unnamed: 0',
 'Institution',
 'Title',
 'Annual Salary',
 'FTE',
 'APPT FTR BASIS',
 'APPT FRACTION',
 'AMOUNT OF SALARY PAID FROM GENERAL FUND',
 '%',
 'FTE.1',
 'Time',
 'PAY_FRCTN',
 'Last Name',
 'First Names',
 'Department',
 'FT/PT',
 'Contract Length',
 'Degree',
 'Highest Education Level',
 'Type',
 'Effective Date',
 'Hire Date',
 'Ethnea',
 'Genni',
 'AdjSalary',
 'PostdocNum',
 'Assistant',
 'Clinical',
 'Faculty',
 'Intern',
 'Scholar',
 'Fellow',
 'Teaching',
 'Researcher',
 'Trainee',
 'Senior',
 'Associate',
 'NIH_grants',
 'NIH_order']

In [56]:
 d.to_csv("Tables/Preprocessed_dataset.csv")

In [60]:
set(d["Department"])

{nan,
 'Molecular & Integrative Physl',
 'Political Science',
 'LSA Biophysics',
 'Beckman Institute',
 'Forest Biomaterials',
 'ROSS SCH Management & Orgs.',
 'Office of Academic Affairs',
 'GEOGRAPHY ENVIRONMENT SPATIAL SCIENCES',
 'Nutritional Sciences',
 'Ctr. for Geospatial Analytics',
 'Human Genetics Department',
 'COS BIOLOGY',
 'Coordinated Science Lab',
 'COMPUTATIONAL MATH SCI AND ENGR CNS',
 'SOM-MED-GASTROENTEROLOGY',
 'Pediatrics-Endocrinology',
 'LSA Kelsey Museum of Arch',
 'ROSS SCH Ctr for Positive Orgs',
 'COE ELECTRICAL ENGINEERING',
 'U of M Energy Institute',
 'CENTER FOR THE STUDY OF CURRICULUM',
 'SOM-MED-INFECTIOUS DISEASE',
 'Mathematics',
 'SOM-SRG-VS CLINICAL',
 'School of Dentistry',
 'LSA Near Eastern Studies',
 'COM IM Pulmonary Division',
 'Chemical & Biomolecular C&G',
 'UMTRI-Engineering Systems',
 'UHS Varsity Village A',
 'Nuclear Eng & Radiological Sci',
 'COM Pathology Hui Lab',
 'Materials Science &Engineering',
 'Radiation Oncology - Physics',
 '